### Context Caching

The Gemini API provides the context caching feature for developers to store frequently used input tokens in a dedicated cache and reference them for subsequent requests, eliminating the need to repeatedly pass the same set of tokens to a model. This feature can help reduce the number of tokens sent to the model, thereby lowering the cost of requests that contain repeat content with high input token counts.

For more information, refer to the [official documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/context-cache/context-cache-overview).


https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/context-caching/intro_context_caching.ipynb

## Pricing
- Models Tokens https://cloud.google.com/vertex-ai/generative-ai/pricing#gemini-models
- Caching Tokens https://cloud.google.com/vertex-ai/generative-ai/pricing#context-caching

In [ ]:
from google.colab import userdata
import json
info = json.loads(userdata.get('JSON_VERTEX_AI'))


In [ ]:
from vertexai.generative_models import GenerativeModel, Part
import vertexai.generative_models as generative_models
from google.oauth2 import service_account
import vertexai
import os
from vertexai import caching
import datetime
from IPython.display import display, Markdown

In [ ]:
vertex_credentials = service_account.Credentials.from_service_account_info(info)
vertexai.init(
    project=userdata.get("PROJECT"),
    location=userdata.get("REGION"),
    credentials=vertex_credentials,
)

In [ ]:
if "GEMINI_API_KEY" not in os.environ:
    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI-API-KEY")
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI-API-KEY")
if "GOOGLE_CLOUD_PROJECT" not in os.environ:
    os.environ["GOOGLE_CLOUD_PROJECT"] = userdata.get("PROJECT")
if "GOOGLE_CLOUD_LOCATION" not in os.environ:
    os.environ["GOOGLE_CLOUD_LOCATION"] = userdata.get("REGION")
if "GOOGLE_GENAI_USE_VERTEXAI" not in os.environ:
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "true"

In [ ]:
def init_model(model: str = "gemini-2.0-flash-001"):
    """Initializes the generative model with the specified model name.
    Args:
        model (str): The name of the model to use
        Returns:
        GenerativeModel: The initialized generative model
    """
    return GenerativeModel(
        model_name=model,
        system_instruction=[
            """You a helpful agent spezialized in tasks with audio files."""
        ],
        safety_settings={
            generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
            generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
            generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
            generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        },
        generation_config={
            "max_output_tokens": 8192,
            "temperature": 1,
            "top_p": 0.95,
        },
    )

In [ ]:
import time
from contextlib import contextmanager

@contextmanager
def measure_time():
    start_time = time.perf_counter()
    yield
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time:.4f} seconds")

In [ ]:
model = init_model()

In [ ]:
bucket_name = "gemini-bot-data"  # Replace with your bucket name
destination_blob_folder =     "test_data"
system_instruction = """
Eres un experto en extraer Informacion de documentos PDF

"""

In [ ]:
contents = [
    Part.from_uri(
        uri="gs://gemini-bot-data/test_data/ley_enjuiciamiento_civil.pdf",
        mime_type="application/pdf"
    ),
    Part.from_uri(
        uri="gs://gemini-bot-data/test_data/BAREMO_AMA_BOE_RD_1971-1999.pdf",
        mime_type="application/pdf"
    ),
    Part.from_uri(
        uri="gs://gemini-bot-data/test_data/Baremo 2015.pdf",
        mime_type="application/pdf"
    )
]

In [ ]:
pdf= Part.from_uri(
        uri="gs://gemini-bot-data/test_data/ley_enjuiciamiento_civil.pdf",
        mime_type="application/pdf"
    )

In [ ]:
with measure_time():
  response = model.generate_content(
      [pdf, "Que nos dice la Ley de Enjuiciamiento civil 1n el articulo 142, Dame una transcripcion completa."],
  )
  print(response.usage_metadata)

prompt_token_count: 92919
candidates_token_count: 195
total_token_count: 93114
prompt_tokens_details {
  modality: TEXT
  token_count: 39
}
prompt_tokens_details {
  modality: DOCUMENT
  token_count: 92880
}
candidates_tokens_details {
  modality: TEXT
  token_count: 195
}

Elapsed time: 37.2371 seconds


In [ ]:
Markdown(response.text)

Claro. Aquí tienes la transcripción del Artículo 142 de la Ley de Enjuiciamiento Civil (LEC):

**Artículo 142. Lengua oficial.**

1.  En todas las actuaciones judiciales, los Jueces, Magistrados, Fiscales, Secretarios Judiciales y demás funcionarios de Juzgados y Tribunales usarán el castellano, lengua oficial del Estado.

2.  Los Jueces, Magistrados, Secretarios Judiciales, Fiscales y demás funcionarios de Juzgados y Tribunales podrán usar también la lengua oficial propia de la Comunidad Autónoma, si ninguna de las partes se opusiere, alegando desconocimiento de ella que pudiere producir indefensión.

3.  Las partes, sus procuradores y abogados, así como los testigos y peritos, podrán utilizar la lengua que sea también oficial en la Comunidad Autónoma en cuyo territorio tengan lugar las actuaciones judiciales, tanto en manifestaciones orales como escritas.

In [ ]:
content_cache = caching.CachedContent.create(
    model_name="gemini-2.0-flash-001",
    system_instruction=system_instruction,
    contents=contents,
    ttl=datetime.timedelta(minutes=60),
)

print(content_cache.name)

6621707623211204608


In [ ]:
content_cache.up

In [ ]:
content_cache.update_time

In [ ]:
content_cache.to_dict()

{'name': 'projects/124210009832/locations/europe-west4/cachedContents/6621707623211204608',
 'model': 'projects/complete-tube-421007/locations/europe-west4/publishers/google/models/gemini-2.0-flash-001',
 'createTime': '2025-04-19T15:18:22.173899Z',
 'updateTime': '2025-04-19T15:18:22.173899Z',
 'expireTime': '2025-04-19T16:18:22.166646Z',
 'usageMetadata': {'totalTokenCount': 247954,
  'textCount': 49,
  'imageCount': 961}}

In [ ]:
cached_content = caching.CachedContent(cached_content_name=content_cache.name)
print(cached_content)

<vertexai.caching._caching.CachedContent object at 0x78f111f0d950>: {
  "name": "projects/complete-tube-421007/locations/europe-west4/cachedContents/6621707623211204608",
  "model": "projects/complete-tube-421007/locations/europe-west4/publishers/google/models/gemini-2.0-flash-001",
  "createTime": "2025-04-19T15:18:22.173899Z",
  "updateTime": "2025-04-19T15:18:22.173899Z",
  "expireTime": "2025-04-19T16:18:22.166646Z",
  "usageMetadata": {
    "totalTokenCount": 247954,
    "textCount": 49,
    "imageCount": 961
  }
}


In [ ]:
with measure_time():
  response2 = model.generate_content(
      "Que nos dice la Ley de Enjuiciamiento civil 1n el articulo 142, Dame una transcripcion completa."
  )


  print(response2.usage_metadata)

prompt_token_count: 39
candidates_token_count: 234
total_token_count: 273
prompt_tokens_details {
  modality: TEXT
  token_count: 39
}
candidates_tokens_details {
  modality: TEXT
  token_count: 234
}

Elapsed time: 2.0191 seconds


In [ ]:
Markdown(response2.text)

Aquí tienes la transcripción completa del Artículo 142 de la Ley de Enjuiciamiento Civil (LEC):

**Artículo 142. Señalamiento de vistas y comparecencias.**

1.  Los señalamientos de vistas y comparecencias se harán procurando que transcurra el menor tiempo posible entre la citación y la fecha del acto, y atendiendo a la naturaleza y complejidad del asunto.

2.  Salvo que la ley disponga otra cosa, el señalamiento de las vistas se hará por el Secretario judicial, teniendo en cuenta las agendas programadas por los Jueces y Magistrados. Si no hubiere acuerdo entre éstos, resolverá el Juez o el Presidente del Tribunal.

3.  Al realizar los señalamientos, se evitará, en la medida de lo posible, que en una misma fecha deban comparecer ante el mismo Tribunal las mismas partes, los mismos procuradores o los mismos abogados.

4.  El señalamiento de las vistas se comunicará a todos los que deban o puedan concurrir a ellas, con antelación suficiente y por el medio más rápido posible.

In [ ]:
with measure_time():
  response3 = model.generate_content(
      "De que trata el real Decreto de ministerio de trabajo y asuntos sociales 1971/1999 en su parte dispositiva"
  )


  print(response3.usage_metadata)

prompt_token_count: 40
candidates_token_count: 455
total_token_count: 495
prompt_tokens_details {
  modality: TEXT
  token_count: 40
}
candidates_tokens_details {
  modality: TEXT
  token_count: 455
}

Elapsed time: 2.9181 seconds


In [ ]:
Markdown(response3.text)

El Real Decreto 1971/1999 del Ministerio de Trabajo y Asuntos Sociales, en su parte dispositiva, se centra en la aprobación del **Reglamento del procedimiento para el reconocimiento, declaración y calificación del grado de minusvalía.**

En esencia, este Real Decreto establece las normas y el proceso a seguir para que una persona pueda obtener el reconocimiento oficial de su discapacidad en España.  La parte dispositiva del Real Decreto aborda:

*   **Ámbito de aplicación:** Define a quién se aplica este reglamento, es decir, a todas las personas que soliciten el reconocimiento, declaración y calificación del grado de minusvalía en el territorio español.
*   **Procedimiento:** Detalla los pasos que deben seguir los interesados para solicitar el reconocimiento de la minusvalía, incluyendo la documentación necesaria, los plazos, la evaluación por los Equipos de Valoración y Orientación (EVO), y las posibles reclamaciones.
*   **Criterios de valoración:** Establece los criterios objetivos que se utilizarán para evaluar el grado de minusvalía, basándose en los baremos establecidos en el Real Decreto.
*   **Órganos competentes:**  Especifica qué órganos administrativos son responsables de llevar a cabo el procedimiento, como los Centros de Valoración y Orientación, y las Consejerías de Asuntos Sociales de las Comunidades Autónomas.
*   **Efectos del reconocimiento:** Describe los efectos legales y administrativos que se derivan del reconocimiento de un determinado grado de minusvalía, como el acceso a prestaciones sociales, beneficios fiscales, o medidas de apoyo para la inclusión laboral.
*   **Revisiones:**  Contempla la posibilidad de revisar el grado de minusvalía reconocido, ya sea por mejoría o empeoramiento de la situación de la persona, o por errores en la valoración inicial.

En resumen, el Real Decreto 1971/1999, a través de su reglamento, proporciona el marco legal para garantizar un proceso justo y homogéneo en toda España para el reconocimiento de la discapacidad, lo cual es fundamental para que las personas con discapacidad puedan acceder a sus derechos y a las medidas de protección social que les corresponden.

In [ ]:
with measure_time():
  response4 = model.generate_content(
      "De que trata la ley 35/2015 de 22 septiembre 2015 y que nos dice en el preambulo"
  )


  print(response4.usage_metadata)

prompt_token_count: 44
candidates_token_count: 443
total_token_count: 487
prompt_tokens_details {
  modality: TEXT
  token_count: 44
}
candidates_tokens_details {
  modality: TEXT
  token_count: 443
}

Elapsed time: 2.9166 seconds


In [ ]:
Markdown(response4.text)

La Ley 35/2015, de 22 de septiembre, de reforma del sistema de valoración de los daños y perjuicios causados a las personas en accidentes de circulación, tiene como objetivo principal **actualizar y mejorar el sistema de valoración de los daños y perjuicios sufridos por las víctimas de accidentes de tráfico.**  Busca proporcionar una indemnización más justa y adecuada a las víctimas, teniendo en cuenta la evolución social, económica y médica.

En cuanto al **preámbulo** de la ley, este nos indica las razones y motivaciones que llevaron a la aprobación de la reforma.  A grandes rasgos, el preámbulo nos dice lo siguiente:

*   **Necesidad de actualización:** El sistema anterior (establecido en la Ley 30/1995) necesitaba una actualización debido a la evolución de la medicina, la jurisprudencia y la sociedad.  El objetivo era modernizar el sistema para reflejar mejor la realidad de los daños sufridos por las víctimas.

*   **Mayor protección a las víctimas:**  La reforma busca mejorar la protección de las víctimas, especialmente aquellas con lesiones más graves, asegurando que reciban una indemnización más justa y adecuada a sus necesidades.

*   **Mayor claridad y seguridad jurídica:**  Se buscaba simplificar y aclarar el sistema de valoración, reduciendo la litigiosidad y proporcionando mayor seguridad jurídica tanto a las víctimas como a las aseguradoras.

*   **Adaptación a la realidad social:** La ley se adapta a los cambios demográficos y sociales, incluyendo aspectos como la edad, el género y las circunstancias personales de la víctima en el cálculo de la indemnización.

*   **Incorporación de avances médicos:** Se incorporan los avances médicos en la valoración de las lesiones, permitiendo una evaluación más precisa del daño sufrido por la víctima.

En resumen, el preámbulo de la Ley 35/2015 explica la necesidad de reformar el sistema de valoración de daños y perjuicios para adaptarlo a la realidad actual, proteger mejor a las víctimas, ofrecer mayor seguridad jurídica y incorporar los avances médicos en la valoración de las lesiones.
